In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, box, Polygon
from shapely.ops import unary_union
import os
import folium
import glob
import pyproj
from pyproj import Geod, Transformer
import laspy
import whitebox
from datetime import datetime

# Initialize WhiteboxTools
wbt = whitebox.WhiteboxTools()

# Directory paths
BONA_las_normalized_directory_2019 = "/data/shared/rsdata/lidar/SMfp/NEON/BONA/BONA_2019_smfp_discrete/"
TEAK_las_normalized_directory_2019 = '/data/shared/rsdata/lidar/SMfp/NEON/TEAK/TEAK_2019_smfp_discrete'

BONA_5km_20m_map_dir = "/data/shared/src/allen/icesat/figs/BONA_5km_20m/map_plot/"

BONA_clipped_las_dir = "/data/shared/src/allen/icesat/clipped_las/BONA/clipped_5km_las"

# Define the CRS EPSG code for NEON BONA site in Alaska
BONA_CRS = 32606
TEAK_CRS = 32611

In [4]:
### Functions for creation of shapefiles

def get_date_subfolder():
    current_date = datetime.now()
    return current_date.strftime('%Y_%m_%d')

def calculate_offset_point(lat, lon, bearing, distance_m):
    """Calculate a point offset from the original point by a given bearing and distance."""
    geod = Geod(ellps="WGS84")
    lon2, lat2, _ = geod.fwd(lon, lat, bearing, distance_m)
    return lon2, lat2

def convert_to_utm(lon, lat, crs):
    """Convert latitude and longitude to easting and northing in the given CRS."""
    transformer = Transformer.from_crs("EPSG:4326", crs, always_xy=True)
    print(f"lat: {lat}, long: {lon}")
    easting, northing = transformer.transform(lon, lat)
    print(f"easting: {easting}, northing: {northing}")
    return easting, northing

def get_utm_crs(latitude, longitude):
    """Determine UTM zone and CRS for a given latitude and longitude."""
    utm_zone = int((longitude + 180) // 6) + 1
    hemisphere = '326' if latitude >= 0 else '327'
    crs = f"EPSG:{hemisphere}{utm_zone:02d}"
    return crs

def utm_to_latlon(easting, northing, crs):
    """Convert UTM coordinates to latitude and longitude using the CRS from the shapefile."""
    # Create CRS object
    utm_crs = pyproj.CRS(crs)
    latlon_crs = pyproj.CRS(proj='latlong', ellps='WGS84')
    
    # Create a transformer for converting coordinates
    transformer = Transformer.from_crs(utm_crs, latlon_crs, always_xy=True)
    lon, lat = transformer.transform(easting, northing)
    return lat, lon

def create_rectangle(lat1, lon1, lat2, lon2, width_m):
    """Create a rectangle polygon for the segment from (lat1, lon1) to (lat2, lon2) with a given width in meters."""
    geod = Geod(ellps="WGS84")
    bearing, _, _ = geod.inv(lon1, lat1, lon2, lat2)
    
    left_bearing = (bearing - 90) % 360
    right_bearing = (bearing + 90) % 360
    
    p1_left = calculate_offset_point(lat1, lon1, left_bearing, width_m / 2)
    p1_right = calculate_offset_point(lat1, lon1, right_bearing, width_m / 2)
    p2_left = calculate_offset_point(lat2, lon2, left_bearing, width_m / 2)
    p2_right = calculate_offset_point(lat2, lon2, right_bearing, width_m / 2)

    # Determine CRS based on the first point
    crs = get_utm_crs(lat1, lon1)
    # print(f"crs: {crs}")

    # Convert points to easting and northing in the determined CRS
    p1_left_utm = convert_to_utm(*p1_left, crs)
    p1_right_utm = convert_to_utm(*p1_right, crs)
    p2_left_utm = convert_to_utm(*p2_left, crs)
    p2_right_utm = convert_to_utm(*p2_right, crs)
    
    # Create and return the polygon in easting and northing along with the CRS
    polygon = Polygon([p1_left_utm, p2_left_utm, p2_right_utm, p1_right_utm, p1_left_utm])
    return polygon, crs

def get_las_bounding_box(las_file):
    """Get the bounding box of a LAS/LAZ file."""
    with laspy.open(las_file) as las:
        # Get the header to extract bounds
        header = las.header
        min_x, min_y, min_z = header.min
        max_x, max_y, max_z = header.max

        # Create a bounding box with the min and max coordinates
        bounding_box = {
            'min_x': min_x,
            'min_y': min_y,
            'max_x': max_x,
            'max_y': max_y
        }
        return bounding_box

def get_las_bounding_boxes(directory):
    """Get bounding boxes for all LAS/LAZ files in the specified directory."""
    las_files = glob.glob(os.path.join(directory, '*.laz'))
    laz_files = glob.glob(os.path.join(directory, '*.las'))
    all_files = las_files + laz_files
    bounding_boxes = {}

    for las_file in all_files:
        bbox = get_las_bounding_box(las_file)
        bounding_boxes[las_file] = box(bbox['min_x'], bbox['min_y'], bbox['max_x'], bbox['max_y'])
    return bounding_boxes

def get_shapefile_bounding_box(shapefile):
    """Get the bounding box of a shapefile."""
    gdf = gpd.read_file(shapefile)
    bounds = gdf.total_bounds  # returns (minx, miny, maxx, maxy)
    
    # Create a bounding box with the min and max coordinates
    bounding_box = {
        'min_x': bounds[0],
        'min_y': bounds[1],
        'max_x': bounds[2],
        'max_y': bounds[3]
    }
    return bounding_box

def get_shapefile_bounding_boxes(directory):
    """Get bounding boxes for all shapefiles in the specified directory."""
    shapefiles = glob.glob(os.path.join(directory, '*.shp'))
    bounding_boxes = {}

    for shapefile in shapefiles:
        bbox = get_shapefile_bounding_box(shapefile)
        bounding_boxes[shapefile] = box(bbox['min_x'], bbox['min_y'], bbox['max_x'], bbox['max_y'])
    return bounding_boxes

def check_shapefiles_within_las_multi(shapefile_dir, las_dir):
    """Check which shapefiles are fully within the combined bounding box of LAS files."""
    las_bboxes = get_las_bounding_boxes(las_dir)
    shapefile_bboxes = get_shapefile_bounding_boxes(shapefile_dir)
    
    # Compute the combined bounding box of all LAS files
    combined_las_bbox = unary_union(list(las_bboxes.values()))
    combined_las_bbox_bounds = combined_las_bbox.bounds
    
    results = {}
    
    for shapefile, shp_bbox in shapefile_bboxes.items():
        # Extract just the filename from the full path
        shapefile_name = os.path.basename(shapefile)

        containing_las_files = []
        
        for las_file, las_bbox in las_bboxes.items():
            if las_bbox.intersects(shp_bbox):
                las_file_name = os.path.basename(las_file)
                containing_las_files.append(las_file_name)
        
        # Check if the shapefile's bbox is within the combined bounding box of the LAS files
        fully_within = combined_las_bbox.contains(shp_bbox)
        # Save results including bounding box information
        results[os.path.basename(shapefile_name)] = {
            'las_files': containing_las_files,
            'fully_within': fully_within,
            'shapefile_bbox': shp_bbox.bounds,  # Bounding box of the shapefile
            'combined_las_bbox': combined_las_bbox.bounds  # Bounding box of the combined LAS area
        }
    
    print(type(shp_bbox))
    print(type(combined_las_bbox))
    return results, combined_las_bbox_bounds

def format_results(results):
    """Format the results into a more readable structure and export to a file if needed."""
    # Convert the results dictionary into a list of dictionaries for better readability
    formatted_results = []
    
    for shapefile, info in results.items():
        formatted_result = {
            'Shapefile': shapefile,
            'Containing LAS Files': ', '.join(info['las_files']),
            'Fully Within Combined LAS Bounding Box': info['fully_within'],
            'Shapefile Bounding Box (min_x, min_y, max_x, max_y)': info['shapefile_bbox'],
            'Combined LAS Bounding Box (min_x, min_y, max_x, max_y)': info['combined_las_bbox']
        }
        formatted_results.append(formatted_result)
    
    # Create a DataFrame for better readability
    df_results = pd.DataFrame(formatted_results)
    
    # Print the results
    print(df_results.to_string(index=False))
    
    # Optionally, export the results to a CSV file
    csv_path = 'shapefile_check_results.csv'
    df_results.to_csv(csv_path, index=False)
    print(f'Results exported to {csv_path}')
    
def process_segment_csv(input_csv_path, shapefile_dir):
    # Ensure the output directory exists
    date_subfolder = get_date_subfolder()
    shapefile_path = os.path.join(shapefile_dir, date_subfolder)

    if not os.path.exists(shapefile_path):
        os.makedirs(shapefile_path)
    
    # Read the CSV file
    df = pd.read_csv(input_csv_path)

    # Process each row in the DataFrame
    for idx, row in df.iterrows():
        segment_id = row['segment_id']
        start_lat, start_lon = row['lat_start'], row['lon_start']
        end_lat, end_lon = row['lat_end'], row['lon_end']

        print(f"segment_id: {segment_id}")
        print(f"start_lat, start_lon: {start_lat}, {start_lon}")
        print(f"end_lat, end_lon: {end_lat}, {end_lon}")

        rectangle, crs = create_rectangle(start_lat, start_lon, end_lat, end_lon, 10)
        
        # Create a GeoDataFrame with the coordinates in UTM
        gdf = gpd.GeoDataFrame({'segment_id': [segment_id], 'geometry': [rectangle]}, crs=crs)

        shapefile_file_path = os.path.join(shapefile_path, f'segment_{segment_id}.shp')

        # Save the GeoDataFrame to a shapefile
        gdf.to_file(shapefile_file_path, driver='ESRI Shapefile')
        print(f"shapefile saved to: {shapefile_file_path}")

    print("Individual shapefiles creation complete.")
    return shapefile_path, df

In [5]:
### Functions for creation of map plot of las files and segments

def plot_boundaries_from_lasfiles(map_object, las_file_dir, crs_epsg):
    """Plot the bounding boxes of LAS/LAZ files on a Folium map."""
    # Retrieve bounding boxes from LAS files
    las_bboxes = get_las_bounding_boxes(las_file_dir)
    
    # Initialize the transformer for the given CRS
    transformer = Transformer.from_crs(crs_epsg, "EPSG:4326", always_xy=True)

    # Plot each bounding box
    for las_file, bbox in las_bboxes.items():
        # Convert bounding box coordinates to lat/lon
        min_x, min_y = bbox.bounds[0], bbox.bounds[1]
        max_x, max_y = bbox.bounds[2], bbox.bounds[3]
        
        min_lon, min_lat  = transformer.transform(min_x, min_y)
        max_lon, max_lat  = transformer.transform(max_x, max_y)

        # Calculate the center of the bounding box
        center_lon = (min_lon + max_lon) / 2
        center_lat = (min_lat + max_lat) / 2
    
        # Extract just the file name (not the full path)
        file_name = os.path.basename(las_file)

        # Add a rectangle to the map
        folium.Polygon(
            locations=[
                [min_lat, min_lon],
                [min_lat, max_lon],
                [max_lat, max_lon],
                [max_lat, min_lon],
                [min_lat, min_lon]
            ],
            color='blue',
            fill=True,
            fill_opacity=0.3,
            # popup=f"LAS File: {las_file}"
        ).add_to(map_object)

        # Add a marker at the center of the bounding box
        folium.Marker(
            location=[center_lat, center_lon],
            popup=file_name,
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_object)
        
        # Debugging statements
        print(f"LAS File: {las_file}")
        print(f"Bounding Box (EPSG:{crs_epsg}): {bbox}")
        print(f"Converted to Lat/Lon: ({min_lat}, {min_lon}) to ({max_lat}, {max_lon})")

    return map_object

def plot_boundaries_from_shapefiles(m, shp_files, color):
    """Plot boundaries from shapefiles onto the Folium map."""
    for shp_file in shp_files:
        gdf = gpd.read_file(shp_file)

        print(f"Reading in shapefile: {shp_file}")

        # Ensure CRS is in the right format for the transformer
        crs = gdf.crs.to_string()
        print(f"crs: {crs}")
        file_name = os.path.basename(shp_file)

        for _, row in gdf.iterrows():
            geom = shape(row['geometry'])
            if geom.geom_type == 'Polygon':
                # Extract boundary coordinates
                coords = list(geom.exterior.coords)
                
                # Convert to latitude and longitude
                lat_lon_coords = [utm_to_latlon(*coord, crs) for coord in coords]

                # Create a popup with the file name
                popup = folium.Popup(file_name, parse_html=True)

                # Plot the polygon on the Folium map
                folium.Polygon(
                    locations=lat_lon_coords,
                    color='black',
                    weight=2,
                    fill=True,
                    fill_color=color,
                    popup=popup  # Add the popup to the polygon
                ).add_to(m)
                
    return m

def plot_segments(segements_df, las_dir, CRS, shapefile_path, map_dir):

    # Calculate mean latitude and longitude for centering the map
    mean_latitude = segements_df[['lat_start', 'lat_end']].mean().mean()
    mean_longitude = segements_df[['lon_start', 'lon_end']].mean().mean()

    date_subfolder = get_date_subfolder()

    # Initialize the map at the mean coordinates
    m = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=12)

    # Get all shapefiles in the specified directory
    shp_files = glob.glob(os.path.join(shapefile_path, '*.shp'))

    # Plot boundaries from LAS files on the map
    plot_boundaries_from_lasfiles(m, las_dir, CRS)

    # Plot shapefile boundaries on the map
    plot_boundaries_from_shapefiles(m, shp_files, 'blue')

    # Save the map as an HTML file in the output directory
    map_plot_save_dir = os.path.join(map_dir, date_subfolder)
    os.makedirs(map_plot_save_dir, exist_ok=True)

    map_save_path = os.path.join(map_plot_save_dir, "BONA_segments_within_LAS_tiles.html")
    m.save(map_save_path)

    print(f"Map saved at {map_save_path}")

    return m

In [6]:
### Functions for clipping las files to shapefiles

def clip_lidar_with_shapefile(input_las_file, shapefile, output_las_file):
    """ Clip a .las file with a shapefile using WhiteboxTools. """
    
    # Set the working directory
    wbt.set_working_dir(os.path.dirname(input_las_file))
    
    # Perform the clipping
    wbt.clip_lidar_to_polygon(
        i=input_las_file,
        polygons=shapefile,
        output=output_las_file
    )
    
    print(f"Clipped {input_las_file} with {shapefile} to {output_las_file}.")

def clip_las_to_shp_multi(results, shapefile_dir, las_dir, output_dir):
    """Clip LAS files based on shapefile results."""
    
    date_subfolder = get_date_subfolder()
    date_dir = os.path.join(output_dir, date_subfolder)
    # Create the directory for the output file if it does not exist
    if not os.path.exists(date_dir):
        os.makedirs(date_dir)
        print(f"Created output directory: {date_dir}")
    
    for shapefile, info in results.items():
        shapefile_path = os.path.join(shapefile_dir, date_subfolder, shapefile)
        containing_las_files = info['las_files']
        output_file = os.path.join(date_dir, f"clipped_{shapefile}.las")

        try:
            if len(containing_las_files) == 1:
                # Only one LAS file - Clip directly
                las_file = os.path.join(las_dir, containing_las_files[0])
                
                # Perform clipping
                clip_lidar_with_shapefile(las_file, shapefile_path, output_file)
                print(f"Clipped {las_file} to {output_file} using {shapefile_path}")

            else:
                # Waiting to figure out how to use LASTools on server
                print("LASMerge is not yet implemented")

        except Exception as e:
            print(f"Error processing {shapefile}: {e}")

    print("Processing complete.")

In [7]:
BONA_input_file_3 = "/data/shared/src/allen/icesat/input_csv/10km_segment_coordinates.csv"

# Ensure the output directory exists
BONA_shapefile_dir_2 = '/data/shared/src/allen/icesat/shapefiles/BONA_5km_segment_shapefiles'

BONA_shpfile_with_date, BONA_df = process_segment_csv(BONA_input_file_3, BONA_shapefile_dir_2)

FileNotFoundError: [Errno 2] No such file or directory: '/data/shared/src/allen/icesat/input_csv/10km_segment_coordinates.csv'

In [24]:
bbox_results_3, full_las_rect_bounds = check_shapefiles_within_las_multi(BONA_shpfile_with_date, BONA_las_normalized_directory_2019)

print (len(bbox_results_3))
format_results(bbox_results_3)

<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.multipolygon.MultiPolygon'>
77
           Shapefile                                                  Containing LAS Files  Fully Within Combined LAS Bounding Box                             Shapefile Bounding Box (min_x, min_y, max_x, max_y) Combined LAS Bounding Box (min_x, min_y, max_x, max_y)
segment_362209.0.shp NEON_D19_BONA_DP1_480000_7222000_classified_point_cloud_colorized.las                                    True    (480379.2971396219, 7222001.294679024, 480391.2497042588, 7222022.192628116)         (468076.95, 7221167.63, 484315.24, 7235887.23)
segment_362345.0.shp NEON_D19_BONA_DP1_476000_7224000_classified_point_cloud_colorized.las                                    True  (476867.15513057786, 7224365.920146064, 476879.43153851276, 7224387.463135251)         (468076.95, 7221167.63, 484315.24, 7235887.23)
segment_639670.0.shp                                                                                   

In [20]:
plot_segments(BONA_df, BONA_las_normalized_directory_2019, BONA_CRS, BONA_shpfile_with_date, BONA_5km_20m_map_dir)

LAS File: /data/shared/rsdata/lidar/SMfp/NEON/BONA/BONA_2019_smfp_discrete/NEON_D19_BONA_DP1_468000_7221000_classified_point_cloud_colorized.las
Bounding Box (EPSG:32606): POLYGON ((468999.99 7221316.23, 468999.99 7221999.99, 468081.96 7221999.99, 468081.96 7221316.23, 468999.99 7221316.23))
Converted to Lat/Lon: (65.11386287622793, -147.67975622146054) to (65.12008498384039, -147.66035885086796)
LAS File: /data/shared/rsdata/lidar/SMfp/NEON/BONA/BONA_2019_smfp_discrete/NEON_D19_BONA_DP1_468000_7222000_classified_point_cloud_colorized.las
Bounding Box (EPSG:32606): POLYGON ((468999.99 7222000, 468999.99 7222999.99, 468076.95 7222999.99, 468076.95 7222000, 468999.99 7222000))
Converted to Lat/Lon: (65.11999719054455, -147.68001969223542) to (65.12905703033408, -147.6605816538426)
LAS File: /data/shared/rsdata/lidar/SMfp/NEON/BONA/BONA_2019_smfp_discrete/NEON_D19_BONA_DP1_468000_7223000_classified_point_cloud_colorized.las
Bounding Box (EPSG:32606): POLYGON ((468999.99 7223000, 468999.99

In [21]:
clip_las_to_shp_multi(bbox_results_3, BONA_shapefile_dir_2, BONA_las_normalized_directory_2019, BONA_clipped_las_dir)

Created output directory: /data/shared/src/allen/icesat/clipped_las/BONA/clipped_5km_las/2024_10_22
./whitebox_tools --run="ClipLidarToPolygon" --wd="/data/shared/rsdata/lidar/SMfp/NEON/BONA/BONA_2019_smfp_discrete" --input='/data/shared/rsdata/lidar/SMfp/NEON/BONA/BONA_2019_smfp_discrete/NEON_D19_BONA_DP1_480000_7222000_classified_point_cloud_colorized.las' --polygons='/data/shared/src/allen/icesat/shapefiles/BONA_5km_segment_shapefiles/2024_10_22/segment_362209.0.shp' --output='/data/shared/src/allen/icesat/clipped_las/BONA/clipped_5km_las/2024_10_22/clipped_segment_362209.0.shp.las' -v --compress_rasters=False

*********************************
* Welcome to ClipLidarToPolygon *
* Powered by WhiteboxTools      *
* www.whiteboxgeo.com           *
*********************************
Reading data...
Performing clip...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
P

In [10]:
TEAK_las_dir_2023 = "C:\\Users\\allen\\OneDrive\\Desktop\\Work\\data\\Tree Segmentation\\TEAK\\NEON_lidar-point-cloud-line\\NEON.D17.TEAK.DP1.30003.001.2023-07.basic.20240409T221345Z.PROVISIONAL"
TEAK_input_csv = "C:\\Users\\allen\\OneDrive\\Desktop\\Work\\Scripts\\small footprint\\TEAK_10km_segment_coordinates_2023.csv"
TEAK_shapefile_path = "C:\\Users\\allen\\OneDrive\\Desktop\\Work\\Scripts\\small footprint\\shapefiles\\TEAK"
TEAK_map_dir = "C:\\Users\\allen\\OneDrive\\Desktop\\Work\\Scripts\\small footprint\\figs\\TEAK\\map_plot"

TEAK_shpfile_with_date, TEAK_df = process_segment_csv(TEAK_input_csv, TEAK_shapefile_path)

segment_id: 796282.0
start_lat, start_lon: 37.05415780089032, -119.0234615515899
end_lat, end_lon: 37.05328751939769, -119.02352981938876
lat: 37.054154973822556, long: -119.02340545006416
easting: 320082.7647234383, northing: 4102794.7428062614
lat: 37.05416062793157, long: -119.02351765311981
easting: 320072.8000583451, northing: 4102795.5825740052
lat: 37.053284692329505, long: -119.02347371850365
easting: 320074.63802874804, northing: 4102698.311382028
lat: 37.053290346439354, long: -119.02358592027802
easting: 320064.67336353974, northing: 4102699.151152698
shapefile saved to: C:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint\shapefiles\TEAK\2024_10_22\segment_796282.0.shp
segment_id: 796287.0
start_lat, start_lon: 37.053277987006446, -119.02357846882134
end_lat, end_lon: 37.05239046179894, -119.02362159014145
lat: 37.05327623383036, long: -119.02352229974397
easting: 320070.2977545267, northing: 4102697.4648589403
lat: 37.053279740155915, long: -119.0236346379013
easti

In [11]:
bbox_results_TEAK, full_las_rect_bounds_TEAK = check_shapefiles_within_las_multi(TEAK_shpfile_with_date, TEAK_las_dir_2023)

print (len(bbox_results_TEAK))
format_results(bbox_results_TEAK)

<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.multipolygon.MultiPolygon'>
101
           Shapefile                                                                                                                         Containing LAS Files  Fully Within Combined LAS Bounding Box                              Shapefile Bounding Box (min_x, min_y, max_x, max_y) Combined LAS Bounding Box (min_x, min_y, max_x, max_y)
segment_205277.0.shp                                                                                                                                                                                False  (325937.99814353616, 4097633.2727110437, 325950.4020308411, 4097661.8327974337)     (312753.043, 4090841.789, 325317.986, 4108573.794)
segment_205279.0.shp                                                                                                                                                                                False  (325867.47323239583, 

In [12]:
plot_segments(TEAK_df, TEAK_las_dir_2023, TEAK_CRS, TEAK_shpfile_with_date, TEAK_map_dir)

LAS File: C:\Users\allen\OneDrive\Desktop\Work\data\Tree Segmentation\TEAK\NEON_lidar-point-cloud-line\NEON.D17.TEAK.DP1.30003.001.2023-07.basic.20240409T221345Z.PROVISIONAL\NEON_D17_TEAK_DP1_312000_4091000_classified_point_cloud_colorized.laz
Bounding Box (EPSG:32611): POLYGON ((312999.999 4091286.88, 312999.999 4091999.997, 312810.551 4091999.997, 312810.551 4091286.88, 312999.999 4091286.88))
Converted to Lat/Lon: (36.94906544868688, -119.10228438341026) to (36.95552715204836, -119.1003344959779)
LAS File: C:\Users\allen\OneDrive\Desktop\Work\data\Tree Segmentation\TEAK\NEON_lidar-point-cloud-line\NEON.D17.TEAK.DP1.30003.001.2023-07.basic.20240409T221345Z.PROVISIONAL\NEON_D17_TEAK_DP1_312000_4092000_classified_point_cloud_colorized.laz
Bounding Box (EPSG:32611): POLYGON ((312999.999 4092000.002, 312999.999 4092999.999, 312753.043 4092999.999, 312753.043 4092000.002, 312999.999 4092000.002))
Converted to Lat/Lon: (36.95547811399816, -119.10310654334873) to (36.96453558718338, -119.10

In [15]:
TEAK_las_clipped_path = "C:\\Users\\allen\\OneDrive\\Desktop\\Work\\Scripts\\small footprint\\figs\\TEAK\\clipped_las"
clip_las_to_shp_multi(bbox_results_TEAK, TEAK_shpfile_with_date, TEAK_las_dir_2023, TEAK_las_clipped_path)

Created output directory: C:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint\figs\TEAK\clipped_las\2024_10_22
LASMerge is not yet implemented
LASMerge is not yet implemented
LASMerge is not yet implemented
LASMerge is not yet implemented
.\whitebox_tools.exe --run="ClipLidarToPolygon" --wd="C:\Users\allen\OneDrive\Desktop\Work\data\Tree Segmentation\TEAK\NEON_lidar-point-cloud-line\NEON.D17.TEAK.DP1.30003.001.2023-07.basic.20240409T221345Z.PROVISIONAL" --input='C:\Users\allen\OneDrive\Desktop\Work\data\Tree Segmentation\TEAK\NEON_lidar-point-cloud-line\NEON.D17.TEAK.DP1.30003.001.2023-07.basic.20240409T221345Z.PROVISIONAL\NEON_D17_TEAK_DP1_319000_4097000_classified_point_cloud_colorized.laz' --polygons='C:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint\shapefiles\TEAK\2024_10_22\2024_10_22\segment_205302.0.shp' --output='C:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint\figs\TEAK\clipped_las\2024_10_22\clipped_segment_205302.0.shp.las' -v --compress_rast

In [24]:
TEAK_las_clipped_path_normalized_2023 = "C:\\Users\\allen\\OneDrive\\Desktop\\Work\\Scripts\\small footprint\\figs\\TEAK\\clipped_las_norm"

def process_lidar_directory(input_dir, output_dir, radius=10.0):
    """Normalize LiDAR files in a directory using the top-hat transform."""
    
    # Initialize WhiteboxTools
    wbt = whitebox.WhiteboxTools()

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Iterate through all .las files in the input directory
    for filename in os.listdir(input_dir):
        print(filename)
        if filename.lower().endswith('.las') or filename.lower().endswith('.laz'):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(output_dir, filename)
            
            # Apply the top-hat transform
            try:
                print(f"Processing {input_file}...")
                wbt.lidar_tophat_transform(
                    input_file, 
                    output_file, 
                    radius=radius
                )
                print(f"Saved normalized file to {output_file}")
            except Exception as e:
                print(f"Error processing {input_file}: {e}")

    print("Normalization complete.")

process_lidar_directory(TEAK_las_clipped_path, TEAK_las_clipped_path_normalized_2023)

2024_10_22
clipped_segment_205302.0.shp.las
Processing C:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint\figs\TEAK\clipped_las\clipped_segment_205302.0.shp.las...
.\whitebox_tools.exe --run="LidarTophatTransform" --input='C:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint\figs\TEAK\clipped_las\clipped_segment_205302.0.shp.las' --output='C:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint\figs\TEAK\clipped_las_norm\clipped_segment_205302.0.shp.las' --radius=10.0 -v --compress_rasters=False

***********************************
* Welcome to LidarTophatTransform *
* Powered by WhiteboxTools        *
* www.whiteboxgeo.com             *
***********************************
reading input LiDAR file...
Performing analysis...
Binning points: 0%
Binning points: 1%
Binning points: 2%
Binning points: 3%
Binning points: 4%
Binning points: 5%
Binning points: 6%
Binning points: 7%
Binning points: 8%
Binning points: 9%
Binning points: 10%
Binning points: 11%
Binning points

In [25]:
TEAK_las_normalized_dir_2023 = "C:\\Users\\allen\\OneDrive\\Desktop\\Work\\data\\Tree Segmentation\\TEAK\\NEON_lidar-point-cloud-line\\TEAK_smfp_2023_normalized"
process_lidar_directory(TEAK_las_dir_2023, TEAK_las_normalized_dir_2023)

2023070816_P1C1_SBET_QAQC.pdf
2023070915_P1C1_SBET_QAQC.pdf
2023_TEAK_6_L1_discrete_lidar_processing.pdf
2023_TEAK_6_L3_discrete_lidar_processing.pdf
NEON.D17.TEAK.DP1.30003.001.readme.20231002T180710Z.txt
NEON_D17_TEAK_DP1_312000_4091000_classified_point_cloud_colorized.laz
Processing C:\Users\allen\OneDrive\Desktop\Work\data\Tree Segmentation\TEAK\NEON_lidar-point-cloud-line\NEON.D17.TEAK.DP1.30003.001.2023-07.basic.20240409T221345Z.PROVISIONAL\NEON_D17_TEAK_DP1_312000_4091000_classified_point_cloud_colorized.laz...
.\whitebox_tools.exe --run="LidarTophatTransform" --input='C:\Users\allen\OneDrive\Desktop\Work\data\Tree Segmentation\TEAK\NEON_lidar-point-cloud-line\NEON.D17.TEAK.DP1.30003.001.2023-07.basic.20240409T221345Z.PROVISIONAL\NEON_D17_TEAK_DP1_312000_4091000_classified_point_cloud_colorized.laz' --output='C:\Users\allen\OneDrive\Desktop\Work\data\Tree Segmentation\TEAK\NEON_lidar-point-cloud-line\TEAK_smfp_2023_normalized\NEON_D17_TEAK_DP1_312000_4091000_classified_point_clo

KeyboardInterrupt: 

In [17]:
def convert_bbox_to_latlon(bbox, current_crs, target_crs="EPSG:4326"):

    min_x, min_y, max_x, max_y = bbox

    # Create a transformer from the current CRS to WGS84 (lat/lon)
    transformer = Transformer.from_crs(current_crs, target_crs, always_xy=True)

    # Convert the corner points of the bounding box
    min_lon, min_lat = transformer.transform(min_x, min_y)
    max_lon, max_lat = transformer.transform(max_x, max_y)

    return {'min_lon': min_lon, 'min_lat': min_lat, 'max_lon': max_lon, 'max_lat': max_lat}

TEAK_full_las_bbox = convert_bbox_to_latlon(full_las_rect_bounds_TEAK, TEAK_CRS)
TEAK_full_las_bbox

{'min_lon': -119.1028195927263,
 'min_lat': 36.945044447770194,
 'max_lon': -118.96590490937703,
 'max_lat': 37.10720783496839}

In [1]:
# Calculate mean latitude and longitude for centering the map
mean_latitude = TEAK_df[['lat_start', 'lat_end']].mean().mean()
mean_longitude = TEAK_df[['lon_start', 'lon_end']].mean().mean()

# Initialize the map at the mean coordinates
m = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=12)

# Plot boundaries from LAS files on the map
plot_boundaries_from_lasfiles(m, TEAK_las_dir_2023, TEAK_CRS)

min_lon = float(TEAK_full_las_bbox['min_lon'])  # Ensure conversion to float
min_lat = float(TEAK_full_las_bbox['min_lat'])  # Ensure conversion to float
max_lon = float(TEAK_full_las_bbox['max_lon'])  # Ensure conversion to float
max_lat = float(TEAK_full_las_bbox['max_lat'])  # Ensure conversion to float

# Add a rectangle for the bounding box
folium.Rectangle(
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],  # Bottom left and top right corners
    color='red',
    fill=True,
    fill_opacity=0.2,
).add_to(m)

# Optionally, add markers for the corners of the bounding box
corners = [
    (min_lat, min_lon),  # Bottom-left
    (min_lat, max_lon),  # Bottom-right
    (max_lat, max_lon),  # Top-right
    (max_lat, min_lon),  # Top-left
]

for corner in corners:
    folium.Marker(location=corner).add_to(m)

m

NameError: name 'TEAK_df' is not defined